# TESTS

## THIS NOTEBOOK

This notebook runs some tests to verify that our code is working as intended. 

In [2]:
#IMPORTS
#libraries
import numpy as np
import time
import import_ipynb

#notebooks
import blockchain as bc
import helper_functions as helfun
import payoff_matrix as pm

importing Jupyter notebook from blockchain.ipynb
importing Jupyter notebook from helper_functions.ipynb
importing Jupyter notebook from payoff_matrix.ipynb


### DRAW A RANDOM BLOCKCHAIN
Print the miners, blocks, the longest chain and [...]

In [3]:
T = 7 #time horizon
n = 2 #number of players

#SET A SEED
seed = int(np.random.randint(0, 1000, 1))
print("seed:", seed, "\n")
np.random.seed(seed)

#GENERATE A RANDOM VALID BLOCKCHAIN
parents = helfun.pickParents(T, n)
winners = helfun.pickWinners(T, n)

B = bc.Blockchain(n, parents, winners) #build

#PRINT ALL MINERS
for i in range(n):
    B.miners[i].printMiner()
print("")

#PRINT ALL BLOCKS
for t in range(1, T+1): #for each element in the blockchain
    B.sequence[t].printBlock()

#DRAW THE BLOCKCHAIN USING TREELIB
helfun.drawChain(B)
print("longest chains", B.longestchains)

seed: 996 

miner 0 wins stages [2 3 5 7]
miner 1 wins stages [1 4 6]

block 1 mined by miner 1, parent is block 0
block 2 mined by miner 0, parent is block 0
block 3 mined by miner 0, parent is block 1
block 4 mined by miner 1, parent is block 1
block 5 mined by miner 0, parent is block 1
block 6 mined by miner 1, parent is block 4
block 7 mined by miner 0, parent is block 4

b0
├── b1
│   ├── b3
│   ├── b4
│   │   ├── b6
│   │   └── b7
│   └── b5
└── b2

longest chains [[0, 1, 4, 6], [0, 1, 4, 7]]


### PAYOFF CALCULATION

In [4]:
#PRINT THE NUMBER OF BLOCKS EACH MINER MINED
#we use the random blockchain generated above
for m in range(2):
    print(f"\nthe number of blocks mined by miner {m} in the chain ending in...")
    for t in range(1, T+1):
        print(f"...block {t} is {B.getPayoff(m, t)}")
print("")

#'TEST' THE FUNCTION 'finalExpectedPayoff()'
#calculates the expected payoff at the end of the game BEFORE nature chooses a the 'winning' chain
for m in range(2):
    print(f"final expected payoff for miner {m} is", pm.finalExpectedPayoff(B, m))


the number of blocks mined by miner 0 in the chain ending in...
...block 1 is 0
...block 2 is 1
...block 3 is 1
...block 4 is 0
...block 5 is 1
...block 6 is 0
...block 7 is 1

the number of blocks mined by miner 1 in the chain ending in...
...block 1 is 1
...block 2 is 0
...block 3 is 1
...block 4 is 2
...block 5 is 1
...block 6 is 3
...block 7 is 2

final expected payoff for miner 0 is 0.5
final expected payoff for miner 1 is 2.5


### BLOCKCHAIN EXTENSION

In [5]:
m = 0 #index of the miner who wins the next stage, arbitrarily chosen
t = T #index of the block the winning miner mines at, also arbitrarily chosen

B_ext = bc.ExtendedBlockchain(B, m, t)

print("extended chain:")
for i in range(n):
    B_ext.miners[i].printMiner()
helfun.drawChain(B_ext)

extended chain:
miner 0 wins stages [2 3 5 7 8]
miner 1 wins stages [1 4 6]

b0
├── b1
│   ├── b3
│   ├── b4
│   │   ├── b6
│   │   └── b7
│   │       └── b8
│   └── b5
└── b2



### PAYOFF MATRIX AND STRATEGIES
Print a payoff matrix and calculate the strategies for each player given that payoff matrix.
By default, it calculates [...]

The reader may experiment with different blockchains. Some examples are given below for blockchain with two, three, and four blocks. The program will handle the payoff calculation. Any payoff matrix can be calculated with relative ease. Please be aware that the program will take a considerable amount of time when $T$ is much larger than the number of blocks in the specified blockchain.

In [10]:
#define variables
T = 4 #time horizon of the game
n = 2 #number of players

#EXAMPLES
#BLOCKCHAIN WITH TWO BLOCKS
parents = np.array([0])
winners = np.array([0]) #miner 0 wins the first stage

#BLOCKCHAIN WITH THREE BLOCKS
#parents = np.array([0, 0]) #both blocks b1 and b2 are appended to the genesis block b0
#winners = np.array([0, 1]) #miner 0 wins the first stage, miner 1 the second

#BLOCKCHAIN WITH FOUR BLOCKS
#parents = np.array([0, 0, 2]) #blocks b1 and b2 are appended to the genesis block b0, b3 is appended to b2
#winners = np.array([0, 1, 1]) #miner 0 wins the first stage, miner 1 the second and third

t = len(parents) + 1 #current stage
print(t)

### ASSERTIONS ###---------------------------------------------------------------------------------------
assert winners.shape[0] == parents.shape[0], "arrays 'parents' and 'winners' must be of same length"
for s in range(t-1):
    assert parents[s] < s+1, "the parent of a block must have a strictly lower index"
###------------------------------------------------------------------------------------------------------


print('T =', T)

B = bc.Blockchain(n, parents, winners)

#PRINT MINERS
for i in range(n): 
    B.miners[i].printMiner()
helfun.drawChain(B)

start = time.process_time()
M = pm.intermediatePayoffMatrix(B, T, n)

end = time.process_time()

print(f"strategy tuple in stage {t}:", pm.getStrategies(B, T, n)[0])

print("\npayoff matrix:\n\n", M)

print('\nthe program took', round((end - start), 0), "seconds to terminate\n")



2
T = 4
miner 0 wins stages [1]
miner 1 wins stages []

b0
└── b1

strategy tuple in stage 2: (1, 1)

payoff matrix:

 [[[1.6875 1.1875]
  [2.     1.5   ]]

 [[2.1875 1.1875]
  [2.5    1.5   ]]]

the program took 0.0 seconds to terminate

